In [1]:
import spacy
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# Data Collection

In [2]:
posPath='imdb/train/trial'
arr = []
for filename in tqdm(os.listdir(posPath)):
    path= os.path.join(posPath,filename)
    with open(path) as f:
        review = f.readlines()
        arr.append(review)
        
posArr = np.array(arr)
posArr = np.insert(posArr, 1, 1, axis=1)

100%|██████████| 2/2 [00:00<00:00, 214.69it/s]


In [4]:
negPath='imdb/train/trial'
arr = []
for filename in tqdm(os.listdir(negPath)):
    #print(filename)
    #path='imdb/train/neg/0_3.txt'
    path= os.path.join(negPath,filename)
    with open(path) as f:
        review = f.readlines()
        arr.append(review)

negArr = np.array(arr)
negArr = np.insert(negArr, 1, 0, axis=1)

100%|██████████| 2/2 [00:00<00:00, 299.57it/s]


In [4]:
print(posArr.shape)
print(negArr.shape)

(12500, 2)
(12500, 2)


In [5]:
df1 = pd.DataFrame(data = posArr, columns = ['review','label'])
df2 = pd.DataFrame(data = negArr, columns = ['review','label'])
reviews_df = df1.append(df2)
print(df1.head(2))
print(df2.head(2))
reviews_df.head()

                                              review label
0  Bromwell High is a cartoon comedy. It ran at t...     1
1  If you like adult comedy cartoons, like South ...     1
                                              review label
0  Bromwell High is a cartoon comedy. It ran at t...     0
1  If you like adult comedy cartoons, like South ...     0


,review,label
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,"If you like adult comedy cartoons, like South ...",1
0,Bromwell High is a cartoon comedy. It ran at t...,0
1,"If you like adult comedy cartoons, like South ...",0


In [6]:
reviews_df.groupby('label').count()

,review
label,
0,2
1,2


In [6]:
reviews_df['vec'] = 0
reviews_df.iloc[0,1]

'1'

In [9]:
reviews_df.head()

,review,label
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,"If you like adult comedy cartoons, like South ...",1
0,Bromwell High is a cartoon comedy. It ran at t...,0
1,"If you like adult comedy cartoons, like South ...",0


In [43]:
reviews_df.review.values[0]
posArr

array([['Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!',
        '1'],
       ["If you like adult comedy cartoons, like South Park, then this is nearly a similar format about the small adventures of three teenage girls at Bromwell High

# Pre Processing

In [7]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(reviews_df.label)

In [59]:
lbl_enc.classes_

array(['0', '1'], dtype=object)

In [10]:
from sklearn.model_selection import train_test_split
xtrain, xvalid, ytrain, yvalid = train_test_split(reviews_df.review, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.3, shuffle=True)

In [12]:
xtrain.shape
#xvalid.shape

(2,)

In [44]:
xtrain[0]
posArr[0]

array(['Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!',
       '1'], dtype='<U806')

In [45]:
arr = ['This is an example of count vectorizer bro. So you better put attention bro.', 'This is the second sentence bro.']
arr

['This is an example of count vectorizer bro. So you better put attention bro.',
 'This is the second sentence bro.']

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(arr)

In [47]:
features.toarray()

array([[1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0]])

In [48]:
words = vectorizer.get_feature_names()
words
pd.DataFrame(features.todense(), columns=words)

,an,attention,better,bro,count,example,is,of,put,second,sentence,so,the,this,vectorizer,you
0,1,1,1,2,1,1,1,1,1,0,0,1,0,1,1,1
1,0,0,0,1,0,0,1,0,0,1,1,0,1,1,0,0


In [54]:
tfv = TfidfVectorizer()
feature= tfv.fit_transform(arr)

In [58]:
# print(tfv.get_feature_names())
# feature.toarray()
words = tfv.get_feature_names()
words
pd.DataFrame(feature.todense(), columns=words)

,an,attention,better,bro,count,example,is,of,put,second,sentence,so,the,this,vectorizer,you
0,0.276951,0.276951,0.276951,0.394106,0.276951,0.276951,0.197053,0.276951,0.276951,0.000000,0.000000,0.276951,0.000000,0.197053,0.276951,0.276951
1,0.000000,0.000000,0.000000,0.334712,0.000000,0.000000,0.334712,0.000000,0.000000,0.470426,0.470426,0.000000,0.470426,0.334712,0.000000,0.000000


In [50]:
tfv = TfidfVectorizer(max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
feature= tfv.fit_transform(arr)

In [53]:
tfv.get_feature_names()

['attention',
 'attention bro',
 'better',
 'better attention',
 'better attention bro',
 'bro',
 'bro better',
 'bro better attention',
 'count',
 'count vectorizer',
 'count vectorizer bro',
 'example',
 'example count',
 'example count vectorizer',
 'second',
 'second sentence',
 'second sentence bro',
 'sentence',
 'sentence bro',
 'vectorizer',
 'vectorizer bro',
 'vectorizer bro better']

In [52]:
feature.toarray()

array([[0.23937921, 0.23937921, 0.23937921, 0.23937921, 0.23937921,
        0.28837731, 0.23937921, 0.23937921, 0.23937921, 0.23937921,
        0.23937921, 0.23937921, 0.23937921, 0.23937921, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.23937921,
        0.23937921, 0.23937921],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.30321606, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.4261596 ,
        0.4261596 , 0.4261596 , 0.4261596 , 0.4261596 , 0.        ,
        0.        , 0.        ]])

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [37]:
xtrain_tfv.toarray()
xtrain_tfv.todense()

matrix([[0.43026606, 0.43026606, 0.1803072 , 0.47050045, 0.1803072 ,
         0.30528663, 0.1803072 , 0.43026606, 0.1803072 ],
        [0.27022684, 0.27022684, 0.27022684, 0.27022684, 0.64484079,
         0.27022684, 0.27022684, 0.27022684, 0.27022684]])

In [54]:
from sklearn.linear_model import LogisticRegression
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
y_pred = clf.predict_proba(xvalid_tfv)

clf.score(xvalid_tfv,yvalid)

0.8834666666666666

In [55]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
confusion_matrix(yvalid, clf.predict(xvalid_tfv))

array([[3242,  508],
       [ 366, 3384]])

In [69]:
yvalid

array([1, 1, 1, ..., 1, 0, 1])

In [71]:
y_pred[:,0]

array([0.08634713, 0.1827365 , 0.32816848, ..., 0.22335579, 0.76034768,
       0.18210268])